# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Study data files
file_loc = "../WeatherPy/output_data/City_Data.csv"

# Read the weather data and the study results
weather_data = pd.read_csv(file_loc)

weather_data.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mossamedes,-15.20,12.15,78.64,49,85,2.24,AO,1589083571
1,Hermanus,-34.42,19.23,57.00,81,0,5.97,ZA,1589083571
2,Punta Arenas,-53.15,-70.92,42.80,93,90,1.54,CL,1589083394
3,Bethel,41.37,-73.41,34.00,69,1,8.05,US,1589083571
4,Bagotville,-28.98,153.42,77.00,57,8,11.41,AU,1589083572


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
location = weather_data[["Latitude", "Longitude"]].astype(float)

# Fill NaN values and convert to float
humidity = weather_data["Humidity"].astype(float)

In [10]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(location, 
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

#save figure
plt.savefig("output_data/vacationheatmap.png")

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#Narrow down the DataFrame to find your ideal weather condition.

#A max temperature lower than 80 degrees but higher than 70.
criteria_1 = weather_data.loc[(weather_data["Max Temp"]>70) & (weather_data["Max Temp"]<80),:]

# #Wind speed less than 10 mph.
criteria_2 = criteria_1.loc[criteria_1["Wind Speed"]<10,:]

# #Zero cloudiness.
criteria_3 = criteria_2.loc[criteria_2["Cloudiness"]==0,:]

criteria_3.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,San Patricio,19.22,-104.70,73.92,79,0,3.38,MX,1589083589
386,Coahuayana Viejo,18.73,-103.68,75.22,82,0,0.96,MX,1589083710
410,Ust'-Ishim,57.69,71.17,71.28,37,0,5.68,RU,1589083720
540,Hanzhong,33.07,107.03,73.09,41,0,2.95,CN,1589083764


In [12]:
criteria_3.dropna(inplace=True)

/Users/HyunsooKim/opt/anaconda3/envs/gmaps/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# from pprint import pprint

# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params = {
#     "location": "-33.93, 18.42", 
#     "radius": "5000",
#     "type": "lodging",
#     "key": g_key,
# }

# response = requests.get(base_url, params=params).json()

# pprint(response)

# result[0]["name"]

In [14]:
hotel_df = criteria_3.copy()

# set up additional columns to hold information
hotel_df['Hotel Name'] = ""

hotel_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
41,San Patricio,19.22,-104.70,73.92,79,0,3.38,MX,1589083589,
386,Coahuayana Viejo,18.73,-103.68,75.22,82,0,0.96,MX,1589083710,
410,Ust'-Ishim,57.69,71.17,71.28,37,0,5.68,RU,1589083720,
540,Hanzhong,33.07,107.03,73.09,41,0,2.95,CN,1589083764,


In [15]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": "5000",
    "type": "lodging",
    "key": g_key,
}

response = requests.get(base_url, params=params).json()

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get locations of the city
    loc_lat=row["Latitude"]
    loc_lon=row["Longitude"]
    locat=f"{loc_lat},{loc_lon}"
    cit = row["City"]

    # add keyword to params dict
    params['location'] = locat

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {cit}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
   
   
    try:
        hotel = results[0]["name"]

        print(f"Closest hotel to {cit} at {loc_lat}, {loc_lon} is {hotel}.")
        
        hotel_df.loc[index, 'Hotel Name'] = hotel

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 41: San Patricio.
Closest hotel to San Patricio at 19.22, -104.7 is Hotel Cabo Blanco.
------------
Retrieving Results for Index 386: Coahuayana Viejo.
Closest hotel to Coahuayana Viejo at 18.73, -103.68 is Hotel Los Arcos.
------------
Retrieving Results for Index 410: Ust'-Ishim.
Closest hotel to Ust'-Ishim at 57.69, 71.17 is Velichko T.p., Gostinitsa.
------------
Retrieving Results for Index 540: Hanzhong.
Closest hotel to Hanzhong at 33.07, 107.03 is Guorui Hotel.
------------


In [16]:
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
41,San Patricio,19.22,-104.70,73.92,79,0,3.38,MX,1589083589,Hotel Cabo Blanco
386,Coahuayana Viejo,18.73,-103.68,75.22,82,0,0.96,MX,1589083710,Hotel Los Arcos
410,Ust'-Ishim,57.69,71.17,71.28,37,0,5.68,RU,1589083720,"Velichko T.p., Gostinitsa"
540,Hanzhong,33.07,107.03,73.09,41,0,2.95,CN,1589083764,Guorui Hotel


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [18]:
# Add marker layer ontop of heat map

heat_layer = gmaps.heatmap_layer(location, 
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=100,
                                 point_radius = 1, 
                                 opacity=1)

marker_layer = gmaps.marker_layer(locations, 
                                  hover_text='', 
                                  label='', 
                                  info_box_content=hotel_info)


fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

plt.savefig("output_data/vacation_with_hotels.png")


# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>